### Agenci AI w Inżynierii Oprogramowania - Maciej Pawłowski i Wiktor Miśkowiec


Framework: Koog - AI Agents Framework by JetBrains
https://github.com/JetBrains/koog

### Demo podstawowego agenta AI z wykorzystaniem Google Gemini 2.5 Pro

In [1]:
import ai.koog.agents.core.agent.AIAgent
import ai.koog.prompt.executor.clients.google.GoogleModels
import ai.koog.prompt.executor.llms.all.simpleGoogleAIExecutor

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY") ?: error("Set GOOGLE_API_KEY env variable")

In [4]:
val agent =
    AIAgent(
        promptExecutor = simpleGoogleAIExecutor(apiKey),
        llmModel = GoogleModels.Gemini2_5Pro,
    )

In [5]:
import kotlinx.coroutines.runBlocking

runBlocking {
    val result = agent.run("Hello! How can you help me?")
    println(result)
}

Hello! That's a great question. I'm a large language model, designed to be a versatile and helpful assistant. Think of me as a multi-tool for information, creativity, and productivity.

Here’s a breakdown of how I can help you:

### 1. Answer Your Questions
You can ask me almost anything. I can act as your personal encyclopedia.
*   **Simple Facts:** "What is the capital of Australia?"
*   **Complex Topics:** "Can you explain the theory of relativity in simple terms?"
*   **How-To Guides:** "How do I bake a sourdough loaf from scratch?"
*   **Definitions:** "What does 'ephemeral' mean?"

### 2. Help You Write and Create
Whether you have writer's block or just need a hand, I can help you generate and refine text.
*   **Drafting:** Emails, essays, reports, cover letters, social media posts.
*   **Creative Writing:** Poems, song lyrics, short stories, scripts.
*   **Content Creation:** Blog post ideas, marketing copy, product descriptions.
*   **Editing:** Proofreading for grammar and spe

### Demo agenta z promptem systemowym

In [9]:
import ai.koog.agents.core.tools.ToolRegistry
import ai.koog.agents.ext.tool.SayToUser

val agent2 = AIAgent(
    promptExecutor = simpleGoogleAIExecutor(apiKey),
    llmModel = GoogleModels.Gemini2_5Pro,
    systemPrompt = "Jesteś pomocnym asystentem AI specjalizującym się w inżynierii oprogramowania. Odpowiadaj śpiewająco.",
    temperature = 0.5,
    toolRegistry = ToolRegistry {
        tool(SayToUser)
    },
)

In [10]:
runBlocking {
    val result = agent2.run("Czym jest język Kotlin?")
    println(result)
}

Agent says: (zanuć wesoło)

Posłuchaj, proszę, opowieści tej,
O języku, co ma wielki wdzięk i styl, hej!
Kotlin się zowie, to jest jego miano,
Przez firmę JetBrains światu został dano!

Na maszynie Javy (JVM) śmiało sobie hula,
I z kodem Javy świetnie się czula!
Jest w stu procentach z nią kompatybilny,
Nowoczesny, zwięzły i bardzo zwinny!

Google go wspiera z całej swojej siły,
By aplikacje na Androida miły
Deweloperzy z uśmiechem tworzyli,
I z "null pointerów" na zawsze drwili!

Więc koduj w Kotlinie, radość z tego miej,
Bo pisze się w nim prościej, lżej, ojej!
(zanuć wesoło)

Posłuchaj, proszę, opowieści tej,
O języku, co ma wielki wdzięk i styl, hej!
Kotlin się zowie, to jest jego miano,
Przez firmę JetBrains światu został dano!

Na maszynie Javy (JVM) śmiało sobie hula,
I z kodem Javy świetnie się czula!
Jest w stu procentach z nią kompatybilny,
Nowoczesny, zwięzły i bardzo zwinny!

Google go wspiera z całej swojej siły,
By aplikacje na Androida miły
Deweloperzy z uśmiechem tworzy

### Demo ze strategy

In [11]:
val promptExecutor = simpleGoogleAIExecutor(apiKey)

In [30]:
import ai.koog.agents.core.tools.SimpleTool
import ai.koog.agents.core.tools.annotations.LLMDescription
import kotlinx.serialization.Serializable
import kotlinx.serialization.KSerializer
import kotlinx.serialization.builtins.serializer

@Serializable
data class WeatherToolsArgs(
    @property:LLMDescription("Date of the day to get weather forecast for (in milliseconds since epoch)")
    val date: Long,
    @property:LLMDescription("City to get weather forecast for")
    val city: String,
)

object WeatherTools : SimpleTool<WeatherToolsArgs>() {



    override val argsSerializer: KSerializer<WeatherToolsArgs> = WeatherToolsArgs.serializer()
    override val name: String = "get_weather_forecast"
    override val description: String = "Tools for getting weather forecast"

    override suspend fun doExecute(args: WeatherToolsArgs): String {
        println("Calling API for $args")
        return "It will be 16 degress and sunny"
    }
}

In [35]:
import ai.koog.agents.core.agent.config.AIAgentConfig
import ai.koog.agents.core.dsl.builder.forwardTo
import ai.koog.agents.core.dsl.builder.strategy
import ai.koog.agents.core.dsl.extension.nodeExecuteTool
import ai.koog.agents.core.dsl.extension.nodeLLMRequest
import ai.koog.agents.core.dsl.extension.nodeLLMSendToolResult
import ai.koog.agents.core.dsl.extension.onAssistantMessage
import ai.koog.agents.core.dsl.extension.onToolCall
import ai.koog.agents.core.feature.handler.agent.AgentCompletedContext
import ai.koog.agents.core.feature.handler.agent.AgentStartingContext
import ai.koog.agents.core.tools.SimpleTool
import ai.koog.agents.core.tools.annotations.LLMDescription
import ai.koog.agents.core.tools.annotations.Tool
import ai.koog.agents.core.tools.reflect.ToolSet
import ai.koog.agents.features.eventHandler.feature.EventHandler
import ai.koog.prompt.dsl.Prompt
import kotlinx.serialization.KSerializer
import kotlinx.serialization.Serializable
import kotlinx.serialization.builtins.serializer

val strategy = strategy<String, String>("Weather forecast strategy") {
    val nodeSendInput by nodeLLMRequest()
    val nodeExecuteTool by nodeExecuteTool()
    val nodeSendToolResult by nodeLLMSendToolResult()

    // Define edges between nodes
    // Start -> Send input
    edge(nodeStart forwardTo nodeSendInput)

    // Send input -> Finish
    edge(
        (nodeSendInput forwardTo nodeFinish)
                transformed { it }
                onAssistantMessage { true }
    )

    // Send input -> Execute tool
    edge(
        (nodeSendInput forwardTo nodeExecuteTool)
                onToolCall { true }
    )

    // Execute tool -> Send the tool result
    edge(nodeExecuteTool forwardTo nodeSendToolResult)

    // Send the tool result -> finish
    edge(
        (nodeSendToolResult forwardTo nodeFinish)
                transformed { it }
                onAssistantMessage { true }
    )
}

val agentConfig = AIAgentConfig(
    prompt = Prompt.build("weather-forecast") {
        system(
            """
                You are weather assistant
                You can get the weather using weather tool.
                """.trimIndent()
        )
    },
    model = GoogleModels.Gemini2_5Flash,
    maxAgentIterations = 10,
)



// Add the tool to the tool registry
val toolRegistry = ToolRegistry {
    tool(WeatherTools)
}

val agent3 = AIAgent(
    promptExecutor = promptExecutor,
    toolRegistry = toolRegistry,
    strategy = strategy,
    agentConfig = agentConfig,
    installFeatures = {
        install(EventHandler) {
            onAgentStarting { eventContext: AgentStartingContext<*> ->
                println("Starting agent: ${eventContext.agent.id}")
            }
            onAgentCompleted { eventContext: AgentCompletedContext ->
                println("Result: ${eventContext.result}")
            }
        }
    }
)

In [36]:
runBlocking {
    // Read the user input and send it to the agent
    val agentResult = agent3.run("Jaka pogoda będzie 26.10.2025 w Katowicach?:")
    println("The agent returned: $agentResult")
}

Starting agent: 22bac00e-6074-48f6-852f-5eabb9bc2014
Calling API for WeatherToolsArgs(date=1761446400000, city=Katowice)
Result: 26.10.2025 w Katowicach będzie 16 stopni Celsjusza i słonecznie.
The agent returned: 26.10.2025 w Katowicach będzie 16 stopni Celsjusza i słonecznie.


### Demo ze structured data (odpowiedzi Typesafe)

In [37]:
import ai.koog.prompt.dsl.prompt
import ai.koog.prompt.structure.executeStructured
import kotlinx.serialization.SerialName

@Serializable
@SerialName("WeatherForecast")
@LLMDescription("Weather forecast for a given location")
data class WeatherForecast(
    @property:LLMDescription("Temperature in Celsius")
    val temperature: Int,
    @property:LLMDescription("Weather conditions (e.g., sunny, cloudy, rainy)")
    val conditions: String,
    @property:LLMDescription("Chance of precipitation in percentage")
    val precipitation: Int
)



val structuredResponse = runBlocking {
    val result = promptExecutor.executeStructured<WeatherForecast>(
        prompt = prompt("structured-data") {
            system(
                """
                You are a weather forecasting assistant.
                When asked for a weather forecast, provide a realistic but fictional forecast.
                """.trimIndent()
            )
            user(
                "What is the weather forecast for Amsterdam?"
            )
        },
        model = GoogleModels.Gemini2_5Flash,
    )

    result.getOrThrow()
}
structuredResponse.structure

WeatherForecast(temperature=12, conditions=Partly cloudy, precipitation=30)

In [38]:
structuredResponse.message.content

{"temperature":12,"conditions":"Partly cloudy","precipitation":30}

Możliwość streamowania rezultatów z LLM zamiast czekać na całą odpowiedź
https://docs.koog.ai/streaming-api/

Możliwość tworzenia reużywalnych nodów któr odpowiadają konkretnym zadaniom
https://docs.koog.ai/custom-nodes/

```kotlin

```

### Przykład reużywalnych niestandardowych nodów

In [44]:
import ai.koog.agents.core.dsl.builder.AIAgentNodeDelegate
import ai.koog.agents.core.dsl.builder.AIAgentSubgraphBuilderBase

strategy<String, String>("strategia") {
    val stringToIndNode by node<String, Int>("node_name") { input ->
        input.length
    }
    val intToStringNode by node<Int, String>("node_name") { input ->
        input.toString()
    }
    val summarizeTextNode by node<String, String>("node_name") { input ->
        llm.writeSession {
            updatePrompt {
                user("Please summarize the following text: $input")
            }
            val response = requestLLMWithoutTools()
            response.content
        }
    }
    edge(nodeStart forwardTo stringToIndNode)
    edge(stringToIndNode forwardTo intToStringNode)
    edge(intToStringNode forwardTo nodeFinish)
}

ai.koog.agents.core.agent.entity.AIAgentGraphStrategy@782268de

Możliwość podziału na subgraphy dla większych zadań
https://docs.koog.ai/custom-subgraphs/
Możliwość zapamiętywania informacji poza sesjami przez agentów
https://docs.koog.ai/agent-memory/

### Przykład architektury MAS z wieloma agentami współpracującymi ze sobą

In [47]:
val masStrategy = strategy<String, String>("MAS-workflow") {
    val managerPlanNode by node<String, String>("manager_plan") { input ->
        llm.writeSession {
            updatePrompt {
                system("""
                You are the ManagerAgent in a Multi-Agent System (MAS) for software modeling.
                Your goal is to create a detailed workflow plan for the given task description.
                Include which agents should be involved and in what order.
                """.trimIndent())

                user("Task description: $input")
            }

            // Zapytanie do LLM-a bez narzędzi (czysty tekst)
            val response = requestLLMWithoutTools()

            // Wynik zwracamy jako output node’a
            response.content
        }
    }

    edge(nodeStart forwardTo managerPlanNode)
    edge(managerPlanNode forwardTo nodeFinish)
}

val agentConfig = AIAgentConfig(
    prompt = Prompt.build("mas-io-workflow") {},
    model = GoogleModels.Gemini2_5Flash,
    maxAgentIterations = 10,
)


val masAgent = AIAgent(
    promptExecutor = promptExecutor,
    strategy = masStrategy,
    agentConfig = agentConfig,
)

In [48]:
runBlocking {
    masAgent.run("Stwórz diagram aktywności dla procesu rejestracji użytkownika w aplikacji mobilnej.")
}

Oto szczegółowy plan działania dla stworzenia diagramu aktywności dla procesu rejestracji użytkownika w aplikacji mobilnej, z uwzględnieniem ról i kolejności zaangażowanych agentów:

---

### **Tytuł Zadania:** Stworzenie diagramu aktywności dla procesu rejestracji użytkownika w aplikacji mobilnej.

### **Cel:** Wygenerowanie jasnego, kompletnego i zgodnego ze standardami UML diagramu aktywności, przedstawiającego krok po kroku proces rejestracji użytkownika w aplikacji mobilnej, włączając w to interakcje z systemem backendowym.

### **Zaangażowani Agenci:**
1.  **ManagerAgent (Ja):** Koordynacja, inicjacja, nadzór i akceptacja finalna.
2.  **Requirement Analyst Agent:** Analiza i uszczegółowienie wymagań biznesowych/funkcjonalnych.
3.  **UML Modeler Agent:** Projektowanie i tworzenie diagramów UML.
4.  **Quality Assurance Agent:** Weryfikacja poprawności, kompletności i zgodności ze standardami.

### **Workflow Plan:**

**Krok 1: Inicjacja i Zbieranie Podstawowych Wymagań**
*   **Agen